# Carpooling and google data exploration

This Notebook evaluates the potentiallity of Google's takeout data with our carpooling algorithm.

##### Notebook by: Keila González-Gómez

In [1]:
pip install --user ipykernel

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Users\keila\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Table of Contents:

1. [Introduction](#introduction)

2. [Required Libraries](#required-libraries)

3. [Read Data](#3-read-data

4. [Data Preparation](#preparation)

5. [Spatial data Preparation](#spatial-preparation)
pip install --user ipykernel
6. [Final Considerations](#considerations)

SyntaxError: invalid syntax (2829323973.py, line 1)

In [ ]:
# Group by tracker to get the top trakers by trips' duration (in minutes)
tracker_trip_duration = df.groupby('tracker_oid').agg({'duration':'sum','distance_by_geom':'sum', 'distance_by_mm_geom':'sum'}).reset_index()
# Transform in hours the trips' duration
tracker_trip_duration['hours'] = tracker_trip_duration['duration']/60


# Group by model from vin number
tracker_model = trackers_info.groupby('model_desc').agg({'tracker_oid':'count','duration':'sum','distance_by_geom':'sum'}).reset_index()
tracker_model

In [ ]:
#Pairplots: Visualize relationships between key numerical variables.
sns.pairplot(trips_filtered[['distance_by_mm_geom', 'avg_speed_by_mm_geom', 'trip_duration_minutes']])

In [ ]:
plt.figure(figsize=(6, 4))
boxplot = plt.boxplot(trips_filtered['trip_duration_minutes'], patch_artist=True)

# Set fill color
for box in boxplot['boxes']:
    box.set(facecolor='#FD9E02')  # Fill color for the box

plt.title("Distribution of Trips' Duration")
plt.ylabel("Trip Duration (minutes)")
plt.show()

In [ ]:
# Heatmap to see densities
map_h = folium.Map(
    location=[df_urban_chile_GranSantiago.geometry.centroid.y, df_urban_chile_GranSantiago.geometry.centroid.x],
    tiles="CartoDB positron",
    zoom_start= 12,
    min_zoom = 8,
    max_zoom = 18)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in gdf_trips_top.geometry]


HeatMap(heat_data,
        radius=12, 
        blur=20, 
        min_opacity=.2,
        ).add_to(map_h)
folium.GeoJson(df_urban_chile_GranSantiago).add_to(map_h)
map_h.save('heatmap_santiago.png')
map_h

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import matplotlib

In [ ]:
x = pd.read_csv(r'C:\Users\keila\Downloads\data1.csv')

In [ ]:
# columns with more than 80% null values
df_telemetrics.columns[df_telemetrics.isnull().mean() > 0.80]

In [ ]:
# keep trips inside AOI 1
gdf_clip_sevilla_trips = gpd.clip(gdf_sevilla_trips, gdf_sevilla_polygon)

In [ ]:
# Define the city name 
city_name = "Andalucia"  # Writing Sevilla brings only the capital and we want the province

# Retrieve graph for the city
# G = ox.graph_from_place(city_name, network_type="all")
G = ox.graph_from_polygon(gdf_sevilla_polygon.geometry[0], network_type="drive")

# Convert the network graph 2 gdf nodes and edges
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G)

1. Define AOI  
    a. The bigger the AOI the slower the calculations  
    b. The bigger more rural areas with low traffic impact will be inserted  
2. Select trips within that AOI  
    a. Several trips will cover the same streets and roads  
    b. Mapmatching and not mapmatched trips would slow analysis  
3. Get the total network from relevant sources  
    a. If the AOI is big this will impact performance  
 4. Compare trips to road network  
    a. Buffer approach or intersection with definite tolerance  
    b.The more lanes the more complicated becomes to define coverage;   

In [ ]:
#!jupyter nbconvert --to slides test_JJPP.ipynb --TemplateExporter.exclude_input=True

In [ ]:
intersecting_trips = gpd.overlay(buffered_roads_7m, 
                                 gdf_chile.to_crs('EPSG:32719'), 
                                 how='intersection', 
                                 keep_geom_type=False)

In [ ]:
osm_f = OSM(get_data('finland'))
nodes_f, edges_f = osm_f.get_network(nodes=True, network_type ='driving')

In [ ]:
fig, ax = plt.subplots()    
# Style
hfont = {'fontname':'Helvetica'}
months = [
    "July",
    "October",
    "September",
    "August",
    "May",
    "December",
    "November",
    "June",
    "April",
    "March",
    "January",
    "February"
]

# Plot
ax.bar(months, most_common_month.values,
        edgecolor='black')
    
# titles
ax.set_ylabel('Number of trips',  **hfont)
#ax.set_title('Trip Distribution in 2023', **hfont, y=1.08)

# Move scientific notation 
t = ax.yaxis.get_offset_text()
t.set_fontsize(10)
t.set_x(0)

# Improve grid settings
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('#525252')
ax.tick_params(axis='x', rotation=45)

# Set grid
ax.set_axisbelow(True)
ax.yaxis.grid(True)
ax.xaxis.grid(False)

# Better visualizatio

**Key insights from the data analysis of the sample telemetrics (telemetrics_202402071517.csv)**:

*Spatial Distribution*:

- The mean distance between points is 277 meters, exhibiting a standard deviation of 315 meters. This indicates significant variability in the separation of points, with the maximum observed distance reaching 5213 meters.

*Temporal Distribution*:

- The mean duration between points is 549 seconds (equivalent to 9.15 minutes), accompanied by a high standard deviation of 6637 seconds. The data distribution appears highly skewed to the right, signifying that the majority of values are concentrated on the longer duration side. To provide a more reliable measure of central tendency, the median duration is 10 seconds.

In [ ]:
#!jupyter nbconvert --to slides test_JJPP.ipynb --TemplateExporter.exclude_input=True

In [ ]:
# Drop columns with 80% or more missing values
def drop_columns_with_high_missing_data(df:pd.DataFrame, threshold:float=0.8)-> pd.DataFrame:
    # Calculate the percentage of missing values for each column
    missing_percentage = df.isnull().mean()

    # Identify columns with missing values exceeding the threshold
    columns_to_drop = missing_percentage[missing_percentage > threshold].index

    # Drop the identified columns from the DataFrame
    df_dropped = df.drop(columns=columns_to_drop)

    return df_dropped

In [ ]:
df_clean = drop_columns_with_high_missing_data(df, 0.8)
df_clean.dtypes

In [ ]:
x.info()

In [ ]:
x.cell_network_type.value_counts()

In [ ]:
x.cell_frequency_band.value_counts()

In [ ]:
x.timing_advance.hist()

In [ ]:
x.timing_advance.describe()

In [ ]:
df_area = gpd.read_file(r'barr2024\barr2024.shp')

In [ ]:
df_area.plot(); 

In [ ]:
x.head(2)

In [ ]:
'214-3-132-20'.split('-')

In [ ]:
x[['pais', 'operador', 'torre', 'celda']] = x.cell_id.str.split('-', expand=True)

In [ ]:
x.head()

In [ ]:
x.pais.value_counts()

In [ ]:
x.operador.value_counts()

In [ ]:
x.torre.nunique()

In [ ]:
x.enodeb.nunique()

In [ ]:
x.ci.nunique()

In [ ]:
x.signal_strength.describe()

In [ ]:
x.connection_type.value_counts()

In [ ]:
df_sample = x.sample(12500)

In [ ]:
gdf = gpd.GeoDataFrame(df_sample, crs='EPSG:4326', geometry = gpd.points_from_xy(df_sample.longitude, df_sample.latitude) )

In [ ]:
m1 = df_area.explore()
gdf.explore(m = m1, color = 'red')

In [ ]:
x.connection_type.describe()

In [ ]:
x.connection_type.unique()

In [ ]:
x.shape

In [ ]:
y = pd.read_csv(r'C:\Users\keila\Downloads\data2.csv')

In [ ]:
y.head()

In [ ]:
y.sector.nunique()

In [ ]:
df_complete = pd.merge(x,y, how='left', left_on = ['ci'], right_on=['ci'])

In [ ]:
df_complete.sector.nunique()

In [ ]:
df_complete

In [ ]:
df_complete.signal_strength.describe()

In [ ]:
df_complete.celda.nunique()

In [ ]:
df_complete.torre.value_counts()

In [ ]:
df_torre_443 = df_complete[df_complete['torre']=='443']

In [ ]:
df_torre_443.sector.value_counts()

In [ ]:
gdf_df_torre_443 = gpd.GeoDataFrame(df_torre_443, crs='EPSG:4326', geometry = gpd.points_from_xy(df_torre_443.longitude, df_torre_443.latitude))

In [ ]:
gdf_df_torre_443.timing_advance.describe()

In [ ]:
gdf_df_torre_443.explore('sector', cmap='Blues')

In [ ]:
df_torre_443_timing = gdf_df_torre_443[gdf_df_torre_443['timing_advance'] < 4]

In [ ]:

df_torre_443_timing.explore('sector', cmap='Blues')

In [ ]:
res = df_torre_443_timing.dissolve("sector").convex_hull

In [ ]:
res.explore()

In [ ]:
df_torre_443_buena_señal

In [ ]:
x.connection_type.value_counts()